# Install

In [1]:
!git clone https://github.com/junyanz/pytorch-CycleGAN-and-pix2pix

Cloning into 'pytorch-CycleGAN-and-pix2pix'...
remote: Enumerating objects: 2513, done.
remote: Total 2513 (delta 0), reused 0 (delta 0), pack-reused 2513
Receiving objects: 100% (2513/2513), 8.20 MiB | 12.58 MiB/s, done.
Resolving deltas: 100% (1575/1575), done.


In [2]:
import os
os.chdir('pytorch-CycleGAN-and-pix2pix/')

In [3]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 17.5 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 37.6 MB/s eta 0:00:0000:0100:01
  Created wheel for visdom: filename=visdom-0.2.4-py3-none-any.whl size=1408196 sha256=a472992c1841ee056341ac62a66604e2f4e81b8299203d5178ecf3aa9a432114
  Stored in directory: /home/jovyan/.cache/pip/wheels/42/29/49/5bed207bac4578e4d2c0c5fc0226bfd33a7e2953ea56356855
Successfully built visdom
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.16.0.dev20230705+cu118
    Uninstalling torchvision-0.16.0.dev20230705+cu118:
      Successfully uninstalled torchvision-0.16.0.dev20230705+cu118


# Datasets

Download one of the official datasets with:

-   `bash ./datasets/download_pix2pix_dataset.sh [cityscapes, night2day, edges2handbags, edges2shoes, facades, maps]`

Or use your own dataset by creating the appropriate folders and adding in the images. Follow the instructions [here](https://github.com/junyanz/pytorch-CycleGAN-and-pix2pix/blob/master/docs/datasets.md#pix2pix-datasets).

In [7]:
#Prepare the p2p dataset
dataroot = "/home/jovyan/data/p2p_msrgb_ndvi"
a = dataroot + "/A"
b = dataroot + "/B"
ab = dataroot + "/AB"
!python datasets/combine_A_and_B.py --fold_A {a} --fold_B {b} --fold_AB {ab}

[fold_A] =  /home/jovyan/data/p2p_msrgb_ndvi/A
[fold_B] =  /home/jovyan/data/p2p_msrgb_ndvi/B
[fold_AB] =  /home/jovyan/data/p2p_msrgb_ndvi/AB
[num_imgs] =  1000000
[use_AB] =  False
[no_multiprocessing] =  False
split = train, use 750/750 images
split = train, number of images = 750
split = test, use 43/43 images
split = test, number of images = 43


# Training

-   `python train.py --dataroot ./datasets/facades --name facades_pix2pix --model pix2pix --direction BtoA`

Change the `--dataroot` and `--name` to your own dataset's path and model's name. Use `--gpu_ids 0,1,..` to train on multiple GPUs and `--batch_size` to change the batch size. Add `--direction BtoA` if you want to train a model to transfrom from class B to A.

In [8]:
dataroot = "/home/jovyan/data/p2p_msrgb_ndvi/AB"

In [9]:
!python train.py --dataroot {dataroot} \
  --name msrgb_ndvi_p2p \
  --model pix2pix \
  --output_nc 1 \
  --direction AtoB \
  --display_id -1 \
  --batch_size 32 \
  --load_size 256 \
  --save_epoch_freq 5 \
  --n_epochs 50 \
  --n_epochs_decay 50 \
  --dataset_mode aligned \
  --use_wandb \
  --wandb_project_name msrgb_ndvi 

----------------- Options ---------------
               batch_size: 32                            	[default: 1]
                    beta1: 0.5                           
          checkpoints_dir: ./checkpoints                 
           continue_train: False                         
                crop_size: 256                           
                 dataroot: /home/jovyan/data/p2p_msrgb_ndvi/AB	[default: None]
             dataset_mode: aligned                       
                direction: AtoB                          
              display_env: main                          
             display_freq: 400                           
               display_id: -1                            	[default: 1]
            display_ncols: 4                             
             display_port: 8097                          
           display_server: http://localhost              
          display_winsize: 256                           
                    epoch: latest        

# Testing

-   `python test.py --dataroot ./datasets/facades --direction BtoA --model pix2pix --name facades_pix2pix`

Change the `--dataroot`, `--name`, and `--direction` to be consistent with your trained model's configuration and how you want to transform images.

> from https://github.com/junyanz/pytorch-CycleGAN-and-pix2pix:
> Note that we specified --direction BtoA as Facades dataset's A to B direction is photos to labels.

> If you would like to apply a pre-trained model to a collection of input images (rather than image pairs), please use --model test option. See ./scripts/test_single.sh for how to apply a model to Facade label maps (stored in the directory facades/testB).

> See a list of currently available models at ./scripts/download_pix2pix_model.sh

In [10]:
!ls checkpoints/
dataroot = "/home/jovyan/data/p2p_msrgb_ndvi/AB"

msrgb_ndvi_p2p


In [13]:
!python test.py --dataroot {dataroot} \
  --name msrgb_ndvi_p2p \
  --model pix2pix \
  --output_nc 1 \
  --direction AtoB \
  --batch_size 32 \
  --load_size 256 \
  --dataset_mode aligned \
  --epoch 100 \
  --phase test

----------------- Options ---------------
             aspect_ratio: 1.0                           
               batch_size: 32                            	[default: 1]
          checkpoints_dir: ./checkpoints                 
                crop_size: 256                           
                 dataroot: /home/jovyan/data/p2p_msrgb_ndvi/AB	[default: None]
             dataset_mode: aligned                       
                direction: AtoB                          
          display_winsize: 256                           
                    epoch: 100                           	[default: latest]
                     eval: False                         
                  gpu_ids: 0                             
                init_gain: 0.02                          
                init_type: normal                        
                 input_nc: 3                             
                  isTrain: False                         	[default: None]
                load